# **CONTEXT**

**Company Name:** Insurance All

**Product/Service:** Offer Health Insurance

**Business Model:** An insurance policy is an arrangement by which a company undertakes to provide a guarantee of compensation for specified loss, damage, illness, or death in return for the payment of a specified premium. A premium is a sum of money that the customer needs to pay regularly to an insurance company for this guarantee.

**Current Situation:** Now they want to offer a car insurance. Last year they did a research with their 380 thousand customers about their interest in getting a car insurance, and the results has been storage in a database together with others customers attributes.
The product team selected 127 thousand new customers to participate in a marketing campaing that will be offering the new car insurance to them. The campaing will be made through phone calls by the sales team.

**Problem:** The problem is that the sales team has capacity of making only 20 thousand calls on the period the campaing will be running.

**Proposed Solution:** Predict the likelihood of the customer of getting the car insurance so that the sales team can prioritize which customer to call to have a better conversion rate.

**Deliverables:** A report answering the following questions:

   **1.** Main insights of the most relevant attributes of the customers interested in getting the car insurance;
    
   **2.** Percentage of interested customers the sales team will be able to call with 20.000 calls;
    
   **3.** Percentage of interested customers the sales team will be able to call with 40.000 calls;
    
   **4.** How many calls the sales team have to make to reach 80% of the interested customers.

# SPRINTS

**SPRINT 1 (05/01 - 12/01)**
- <s>Solution Planning (Done)</s>;
- <s>Data Collection (Done)</s>;
    
**SPRINT 2 (12/01 - 19/01)**
- Descriptive Analysis;
- Business Research (What is a cross-sell);

**SPRINT 3 (19/01 - 26/01)**
- EDA;
- Insight Report;

**SPRINT 4 (26/01 - 02/02)**
- Data Preparation;

**SPRINT 5 (02/02 - 09/02)**
- Implementation of Machine Learning Models;

**SPRINT 6 (09/02 - 16/02)**
- Business Metrics (KPI);
- Translate to business metrics;

**SPRINT 7 (16/02 - 23/02)**
- Deploy of the Model in Production;

**SPRINT 8 (23/02 - 02/03)**
- Access of the data in the Model in Production;

**SPRINT 9 (02/03 - 09/03)**
- Presentation to Business Team;

**SPRINT 10 (09/03 - 16/03)**
- Write an article;
- What I Learned;

# IMPORTS

In [72]:
# Data Manipulation
import pandas as pd
import numpy as np
import statistics as st

# General Utilities
import pickle
import warnings
warnings.filterwarnings('ignore')

# Database Library
import psycopg2


# 1.0. SOLUTION PLANNING

- Build a Machine Learning model to calculate the likelihood of customers getting the car insurance. For that we will do the following steps:
    - Data Collecting: from the Database;
    - Descriptive analysis:  see how the data is. (To see how much work needs to be done to processing data.)
    - Exploratory Data Analisys:  see the behavior of the data (features).
    - Data Preparation: Preparating data to fit into machine learning model;
    - Implementation of machine learning models;
    - Interpretating the metrics of the model;
    - Translating the model metrics to business metrics;
    - Deploy of the model in production;
 

## 1.1. Main insights of the most relevant attributes of the customers interested in getting the car insurance;

- Separete the data from customers that are interested from those that are not interested in getting a car insurance;
- Compare the data between those that are interested and those that are not interested;
- See if there are any insight. (different behavior in datas)

## 1.2. Percentage of interested customers the sales team will be able to call with 20.000 calls;

- Use Machine Learning to predict the total number of customers that are interested in getting a car insurence from those 127 thousand customers that will be participating in the marketing campaing;
- From that, we can calculate the percentage of customers the sales team will be able to reach with 20.000 calls.

## 1.3. Percentage of interested custumers the sales team will be able to call with 40.000 calls;

- Same as 1.2.

## 1.4. How many calls the sales team have to make to reach 80% of the interested custumers.

- After calculating the total number of customers interested in getting car insurance, we can calculate how many calls would be necessary to reach 80% of those customers.

# 2.0. GETTING THE DATA FROM DATABASE

In [2]:
# Credentials of the Database
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username = 'member'
password = 'cdspa'

# Connecting to Database
conn = psycopg2.connect(host=host, port=port, database=database, user=username, password=password)

# Cursor
cursor = conn.cursor()

In [3]:
# Listing Schemas
cursor.execute("select schema_name from information_schema.schemata;")
print (cursor.fetchall())

[('pg_catalog',), ('information_schema',), ('public',), ('pa004',)]


In [4]:
# Showing tables from Database
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print (cursor.fetchall())

[('users',), ('vehicle',), ('insurance',)]


In [5]:
# Selecting to get columns name from tables (users, vehicle, insurance)

# users
query = "SELECT column_name FROM information_schema.columns WHERE table_name ='users'"
cursor.execute(query)
cols_user = list(map(lambda x: x[0], cursor.fetchall()))

# vehicle
query = "SELECT column_name FROM information_schema.columns WHERE table_name ='vehicle'"
cursor.execute(query)
cols_vehicle = list(map(lambda x: x[0], cursor.fetchall()))

# insurance
query = "SELECT column_name FROM information_schema.columns WHERE table_name ='insurance'"
cursor.execute(query)
cols_insurance = list(map(lambda x: x[0], cursor.fetchall()))

# showing column names
cols_all = pd.DataFrame([cols_user, cols_vehicle, cols_insurance])
cols_all.rename(index={0:'user', 1:'vehicle', 2:'insurance'})

,0,1,2,3,4
user,id,gender,age,region_code,policy_sales_channel
vehicle,id,driving_license,vehicle_age,vehicle_damage,None
insurance,id,previously_insured,annual_premium,vintage,response


In [6]:
# Selecting get data from tables (users, vehicle, insurance)

# users
query = "SELECT * FROM pa004.users"
cursor.execute(query)
df_users = pd.DataFrame(cursor.fetchall(), columns=cols_user)

# vehicle
query = "SELECT * FROM pa004.vehicle"
cursor.execute(query)
df_vehicle = pd.DataFrame(cursor.fetchall(), columns=cols_vehicle)

# insurance
query = "SELECT * FROM pa004.insurance"
cursor.execute(query)
df_insurance = pd.DataFrame(cursor.fetchall(), columns=cols_insurance)

In [7]:
# Merge tables to DataFrame
df2 = pd.merge(pd.merge(df_users, df_vehicle), df_insurance)
df2.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,1,Male,44,28.0,26.0,1,> 2 Years,Yes,0,40454.0,217,1
1,2,Male,76,3.0,26.0,1,1-2 Year,No,0,33536.0,183,0
2,3,Male,47,28.0,26.0,1,> 2 Years,Yes,0,38294.0,27,1
3,4,Male,21,11.0,152.0,1,< 1 Year,No,1,28619.0,203,0
4,5,Female,29,41.0,152.0,1,< 1 Year,No,1,27496.0,39,0


# 3.0. Descriptive Analysis

In [16]:
df3 = df2.copy()
df3.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,1,Male,44,28.0,26.0,1,> 2 Years,Yes,0,40454.0,217,1
1,2,Male,76,3.0,26.0,1,1-2 Year,No,0,33536.0,183,0
2,3,Male,47,28.0,26.0,1,> 2 Years,Yes,0,38294.0,27,1
3,4,Male,21,11.0,152.0,1,< 1 Year,No,1,28619.0,203,0
4,5,Female,29,41.0,152.0,1,< 1 Year,No,1,27496.0,39,0


In [9]:
print(f'There are {df3.shape[0]} rows')
print(f'There are {df3.shape[1]} columns')

There are 381109 rows
There are 12 columns


In [22]:
# Description of variables
variables = {'id': 'Unique ID for the customer',
             'gender': '(Male or Female)',
             'age': 'Age of the customer',
             'region_code': 'Unique code for the region of the customer',
             'policy_sales_channel': 'Anonymized Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc.', 
             'driving_license': '0 : Customer does not have DL, 1 : Customer already has DL',
             'vehicle_age': 'Age of the Vehicle',
             'vehicle_damage': "Yes or No",
             'previously_insured': "1 : Customer already has Vehicle Insurance, 0 : Customer doesn't have Vehicle Insurance",
             'annual_premium': 'The amount customer needs to pay as premium in the year',
             'vintage': 'Number of Days, Customer has been associated with the company',
             'response': '1 : Customer is interested, 0 : Customer is not interested'}


pd.set_option('display.max_colwidth', -1);
pd.DataFrame.from_dict(variables, orient='index', columns=['description'])

,description
id,Unique ID for the customer
gender,(Male or Female)
age,Age of the customer
region_code,Unique code for the region of the customer
policy_sales_channel,"Anonymized Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc."
driving_license,"0 : Customer does not have DL, 1 : Customer already has DL"
vehicle_age,Age of the Vehicle
vehicle_damage,Yes or No
previously_insured,"1 : Customer already has Vehicle Insurance, 0 : Customer doesn't have Vehicle Insurance"
annual_premium,The amount customer needs to pay as premium in the year


## 3.1. Checking NA

In [11]:
df3.isna().sum()

id                      0
gender                  0
age                     0
region_code             0
policy_sales_channel    0
driving_license         0
vehicle_age             0
vehicle_damage          0
previously_insured      0
annual_premium          0
vintage                 0
response                0
dtype: int64

No missing values in the data set

## 3.2. Checking Data Types

In [12]:
df3.dtypes

id                      int64  
gender                  object 
age                     int64  
region_code             float64
policy_sales_channel    float64
driving_license         int64  
vehicle_age             object 
vehicle_damage          object 
previously_insured      int64  
annual_premium          float64
vintage                 int64  
response                int64  
dtype: object

## 3.3. Descriptive Statistics

### 3.3.1. Target Attribute

In [124]:
df3['response'].value_counts()

0    334399
1    46710 
Name: response, dtype: int64

In [125]:
df3['response'].value_counts(normalize=True)

0    0.877437
1    0.122563
Name: response, dtype: float64

So there are only 12.25% of the target attribute that has interest in getting a car insurance.

This shows us that the data is unbalanced.

### 3.3.2 Numerical Attributes

In [111]:
# Calculating statistics
descrip_stats_num = df3.describe()

# median
descrip_stats_num.loc['median'] = descrip_stats_num.apply(np.median)

# range
descrip_stats_num.loc['range'] = descrip_stats_num.loc['max'] - descrip_stats_num.loc['min']

# skew
descrip_stats_num.loc['skew'] = descrip_stats_num.skew()

# kurtosis
descrip_stats_num.loc['kurtosis'] = descrip_stats_num.kurtosis()


descrip_stats_num.reindex(['count',
                           'min',
                           '25%',
                           '50%',
                           '75%',
                           'max',
                           'range',
                           'std',
                           'mean',
                           'median',
                           'skew',
                           'kurtosis']).T

,count,min,25%,50%,75%,max,range,std,mean,median,skew,kurtosis
id,381109.0,1.0,95278.0,190555.0,285832.0,381109.0,381108.0,110016.836208,190555.000000,190555.000000,-0.084852,-1.261004
age,381109.0,20.0,25.0,36.0,49.0,85.0,65.0,15.511611,38.822584,37.411292,3.162277,10.999999
region_code,381109.0,0.0,15.0,28.0,35.0,52.0,52.0,13.229888,26.388807,27.194404,3.162278,10.999999
policy_sales_channel,381109.0,1.0,29.0,133.0,152.0,163.0,162.0,54.203995,112.034295,122.517147,3.162276,10.999992
driving_license,381109.0,0.0,1.0,1.0,1.0,1.0,1.0,0.046110,0.997869,1.000000,3.162278,11.000000
previously_insured,381109.0,0.0,0.0,0.0,1.0,1.0,1.0,0.498251,0.458210,0.478231,3.162278,11.000000
annual_premium,381109.0,2630.0,24405.0,31669.0,39400.0,540165.0,537535.0,17213.155057,30564.389581,31116.694791,1.153087,-0.206512
vintage,381109.0,10.0,82.0,154.0,227.0,299.0,289.0,83.671304,154.347397,154.173698,3.162274,10.999978
response,381109.0,0.0,0.0,0.0,0.0,1.0,1.0,0.327936,0.122563,0.061282,3.162278,11.000000


### 3.3.3. Categorical Attributes

In [127]:
descrip_stats_cat = df3.select_dtypes(include='object')
descrip_stats_cat.describe()

,gender,vehicle_age,vehicle_damage
count,381109,381109,381109
unique,2,3,2
top,Male,1-2 Year,Yes
freq,206089,200316,192413
